# Visualization

可能要分成多个图
- 按照version绘图，表示出标记正确的version和标记错误的version以及占比
- 按照API绘图，以API的链接作为区分，表示出正确和错误的占比
- 分别以可否访问endpoint 和 portal/homepage 作为API是否存活的依据，绘制上面两个图

In [ ]:
import matplotlib.pyplot as plt
import json
from crawler.data_objects import SimplifiedApi, VersionVisit, VisitStatus

In [ ]:
# 读取version_status 文件
version_visits = []
for i in range(1, 6):
    text = ''
    with open('version_status0915-' + str(i) + '.txt', 'r', encoding='utf-8') as file:
        version_visits_dicts = json.load(fp=file)
    print(len(text))
    # json 解析
    for visit_dict in version_visits_dicts:
        from_api_dict = visit_dict['from_api']
        from_api = SimplifiedApi(from_api_dict['api_title'], from_api_dict['url'])
        visit_status_list = []
        for status in visit_dict['visit_status']:
            visit_status_list.append(VisitStatus(status['visit_url'], status['visit_label'], status['status_code']))
        version_visits.append(VersionVisit(visit_dict['version_title'], visit_dict['version_url'], from_api, visit_status_list, visit_dict['is_accessible'], visit_dict['is_endpoint_accessbile'], visit_dict['is_homepage_accessible']))

In [ ]:
len(version_visits)

In [ ]:
def show_pie(version_visits):
    # 分成四个图
    # 先统计
    # 1. 按照 endpoint 统计
    # 2. 按照 portal 统计
    apis = set()
    endpoint_accessible_versions = []
    endpoint_accessible_apis = set()
    homepage_accessible_versions = []
    homepage_accessible_apis = set()
    for visit in version_visits:
        # 如果是对象
        from_api_url = visit.from_api.url.strip()
        apis.add(from_api_url)

        for status in visit.visit_status:
            if status.visit_label == 'API Endpoint':
                if status.status_code == 200 or status.status_code == 429:
                    endpoint_accessible_versions.append(visit)
                    endpoint_accessible_apis.add(from_api_url)      # 这就保证了只要有一个version是可访问的，这个API就作为可访问的
            elif status.visit_label == 'API Portal / Home Page':
                if status.status_code == 200 or status.status_code == 429:
                    homepage_accessible_versions.append(visit)
                    homepage_accessible_apis.add(from_api_url)

    # 统计完了

    # 绘图
    version_length = len(version_visits)
    api_length = len(apis) + 1951

    plt.suptitle("Active versions' accessibility")
    p1 = plt.subplot(1, 2, 1)
    plt.title('API Endpoint')
    endpoint_length = len(endpoint_accessible_versions)
    plt.pie(x=[endpoint_length, version_length - endpoint_length], labels=['Accessible', 'Inaccessible'], autopct='%.2f%%')

    p2 = plt.subplot(1, 2, 2)
    plt.title('API Portal / Home Page')
    homepage_length = len(homepage_accessible_versions)
    plt.pie(x=[homepage_length, version_length - homepage_length], labels=['Accessible', 'Inaccessible'], autopct='%.2f%%')
    plt.show()

    plt.suptitle("Active APIs' accessibility")
    p3 = plt.subplot(1, 2, 1)
    plt.title('API Endpoint')
    endpoint_api_length = len(endpoint_accessible_apis)
    plt.pie(x=[endpoint_api_length, api_length - endpoint_api_length], labels=['Accessible', 'Inaccessible'], autopct='%.2f%%')

    p4 = plt.subplot(1, 2, 2)
    plt.title('API Portal / Home Page')
    homepage_api_length = len(homepage_accessible_apis)
    plt.pie(x=[homepage_api_length, api_length - homepage_api_length], labels=['Accessible', 'Inaccessible'], autopct='%.2f%%')
    plt.show()

    print('Judged by API Endpoint, ' + str(endpoint_api_length) + ' of ' + str(api_length) + ' in all are active')
    print('Judged by API Homepage, ' + str(homepage_api_length) + ' of ' + str(api_length) + ' in all are active')

show_pie(version_visits)